In [1]:
import pandas as pd
import os
import time
import numpy as np
import arcgis
from astral import Astral
from datetime import datetime, timedelta

# Preparing the training data
In this notebook, we prepare the training data for use by machine learning algorithms. Note that there is some prep still to be done after this notebook, depending on the particular machine learning algorithm to use, so it's left in a generic format without categorical variables encoded, without scaling, etc. 

## 1. Load/Condition Data from GDB


With the prepwork done in Pro or Arcpy, we can use the ArcGIS Python API to easily load this data into a SpatialDataFrame, which will allow us to do any geospatial manipulations we need to do. In this example, we'll mostly use the ``pandas`` functionality

In [2]:
project_gdb = r'utah.gdb'
collisions_path = os.path.join(project_gdb,'collisions_joined')
road_features_path = os.path.join(project_gdb,'static_features')

## 1.1 Load Data

In [3]:
df = arcgis.features.SpatialDataFrame().from_featureclass(collisions_path)

In [4]:
df.head()

,OBJECTID,Join_Count,TARGET_FID,CRASHID,CRASHDATE,CRASHTIME,RouteName,BeginMilePost,RoadView,CrashSeverity,...,NUMBERVEHICLESINVOLVED,FIRSTHARMFULEVENTID,ROADWAYTYPECD,ROUTEDIRECTION,EXITNUMBER,RAMPID,Latitude,Longitude,segment_id,SHAPE
0,1,1,1,10401151,2010-10-15,19:56:00,0009P,11.64,http://168.178.125.111/virtualnavigator/udot/V...,2,...,2,Other Motor Vehicle in Transport,Mainline,P,None,None,4119389.0,298229.0,28420,"{'x': -12609547.622000001, 'y': 4466906.681199..."
1,2,1,2,10394697,2010-08-20,19:33:00,0009P,11.85,http://168.178.125.111/virtualnavigator/udot/V...,1,...,2,Other Motor Vehicle in Transport,Mainline,P,None,None,4119704.0,298349.0,28426,"{'x': -12609406.6849, 'y': 4467306.825099997, ..."
2,3,1,3,10346333,2010-03-27,21:30:00,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,...,1,Delineator Post,Mainline,P,None,None,4122320.0,299671.0,24673,"{'x': -12607827.8035, 'y': 4470640.231600001, ..."
3,4,1,4,10352753,2010-04-16,13:45:00,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,...,1,Overturn/Rollover,Mainline,P,None,None,4122320.0,299671.0,24673,"{'x': -12607827.8035, 'y': 4470640.231600001, ..."
4,5,1,5,10357506,2010-03-05,9:48:00,0009P,21.00,http://168.178.125.111/virtualnavigator/udot/V...,1,...,1,Animal - Wild,Mainline,P,None,None,4118637.0,309403.0,28596,"{'x': -12595519.862399999, 'y': 4466288.306199..."


In [5]:
rdf = arcgis.features.SpatialDataFrame().from_featureclass(road_features_path)

## 1.2 Remove some bad data

In [6]:
# We don't care about accidents that happened far from known major/minor roads
df = df.dropna(how='any',subset=['segment_id'])

# Convert some types
df['segment_id'] = df.segment_id.astype('int64')
rdf['segment_id'] = rdf.segment_id.astype('int64') 
rdf['station_id'] = rdf.station_id.astype('int64') # Weather statiion ID

## 1.3 Parse Dates

In [7]:
df['CRASHDATE'] = df['CRASHDATE'].astype('str')
df['CRASHTIME'] = df['CRASHTIME'].astype('str')

df['timestamp'] = pd.to_datetime(df['CRASHDATE']+' '+df['CRASHTIME'])

time_index = pd.DatetimeIndex(df.timestamp).floor('1h')
df['hour'] = time_index.hour
df['weekday'] = time_index.weekday
df['month'] = time_index.month
if not os.path.exists('training_data'):
    os.mkdir('training_data')
df.to_csv('training_data/collisions.csv')

## 1.4 Remove/Replace missing data

In [8]:
rdf.loc[pd.isna(rdf.speed_limit),'speed_limit'] = np.median(rdf.speed_limit)
rdf.loc[pd.isna(rdf.aadt),'aadt'] = 0.0 # AKA, unknown, zero, etc. This will help differentiate major/minor roadways

# 2. Expand Feature Set
Once the data is in a DataFrame, it's very easy to manipulate to add certain features. Here we will add a accident count field by doing some joins and aggregations

In [9]:
accident_counts_per_segment = df.groupby('segment_id').size().reset_index(name='accident_counts').set_index('segment_id')

In [10]:
rdf = rdf.set_index('segment_id').join(accident_counts_per_segment)

In [11]:
# If there was no accident, set value to 0, not NaN
rdf.loc[pd.isna(rdf.accident_counts),'accident_counts'] = 0.0

# 3. Select Features

## 3.1 Feature Names
We can do most of our exploratory data analysis in ArcGIS Pro to determine a set of candidate features. Once we build our training set, we can do further EDA to determine the best feature set. Here we select the candidate features that we found in Pro 

In [12]:
print(rdf.columns.tolist())

['OBJECTID_1', 'Join_Count', 'TARGET_FID', 'Join_Count_1', 'TARGET_FID_1', 'Join_Count_12', 'TARGET_FID_12', 'OBJECTID', 'pre_dir', 'street_type', 'suf_dir', 'one_way', 'speed_limit', 'surface_type', 'surface_width', 'aadt', 'sinuosity', 'euclidean_length', 'segment_length', 'at_intersection', 'near_billboard', 'road_orient_approx', 'proximity_to_signal', 'proximity_to_billboard', 'proximity_to_nearest_intersection', 'proximity_to_major_road', 'population_density', 'station_id', 'SHAPE', 'accident_counts']


In [13]:
static_feature_names = [
    'pre_dir', 
    'street_type',
    'suf_dir', 
    'one_way', 
    'speed_limit', 
    'surface_type', 
    'surface_width', 
    'aadt', 
    'sinuosity',          # \
    'euclidean_length',   # |  These together define road curvature
    'segment_length',     # /
    'road_orient_approx', 
    'at_intersection', 
    'proximity_to_billboard',
    'proximity_to_major_road',
    'proximity_to_signal',
    'proximity_to_nearest_intersection',
    'population_density',
    'accident_counts'
]
temporal_feature_names = [
    'hour',
    'weekday',
    'month'
]

## 3.2 Select static features, join to accidents records

In [14]:
features = df\
    .xs(['timestamp','segment_id']+temporal_feature_names,axis=1)\
    .set_index('segment_id')\
    .join(
        rdf.xs(static_feature_names+['station_id'],axis=1),
        how='left'
    ).reset_index()

In [15]:
features.head()

,segment_id,timestamp,hour,weekday,month,pre_dir,street_type,suf_dir,one_way,speed_limit,...,segment_length,road_orient_approx,at_intersection,proximity_to_billboard,proximity_to_major_road,proximity_to_signal,proximity_to_nearest_intersection,population_density,accident_counts,station_id
0,1,2016-11-26 10:13:00,10,5,11,N,,,0,40.0,...,110.198977,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,72572024127
1,1,2017-10-16 16:03:00,16,0,10,N,,,0,40.0,...,110.198977,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,72572024127
2,1,2010-01-16 15:26:00,15,5,1,N,,,0,40.0,...,110.198977,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,72572024127
3,1,2013-12-16 15:48:00,15,0,12,N,,,0,40.0,...,110.198977,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,72572024127
4,2,2015-09-04 21:10:00,21,4,9,N,,,0,40.0,...,87.835598,1.187714,1,887.740871,0.0,16.807568,0.717029,1.906303,7.0,72572024127


## 3.3 Index by timestamp/road

In [16]:
time_index = pd.DatetimeIndex(features.timestamp).floor('1h')
feature_timestamp_series = time_index.to_series()
segment_id = features.segment_id.astype('str')

# Floor to nearest hour
features['timestamp'] = feature_timestamp_series.values

# Index for fast lookup
features['fid'] = feature_timestamp_series.map(lambda x:str(int(x.timestamp()))).values + segment_id.values
features.set_index('fid',inplace=True)

# Set target value
features['target'] = 1

In [17]:
features.head(15)

,segment_id,timestamp,hour,weekday,month,pre_dir,street_type,suf_dir,one_way,speed_limit,...,road_orient_approx,at_intersection,proximity_to_billboard,proximity_to_major_road,proximity_to_signal,proximity_to_nearest_intersection,population_density,accident_counts,station_id,target
fid,,,,,,,,,,,,,,,,,,,,,
14801544001,1,2016-11-26 10:00:00,10,5,11,N,,,0,40.0,...,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,72572024127,1
15081696001,1,2017-10-16 16:00:00,16,0,10,N,,,0,40.0,...,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,72572024127,1
12636540001,1,2010-01-16 15:00:00,15,5,1,N,,,0,40.0,...,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,72572024127,1
13872060001,1,2013-12-16 15:00:00,15,0,12,N,,,0,40.0,...,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,72572024127,1
14414004002,2,2015-09-04 21:00:00,21,4,9,N,,,0,40.0,...,1.187714,1,887.740871,0.0,16.807568,0.717029,1.906303,7.0,72572024127,1
13190904002,2,2011-10-20 06:00:00,6,3,10,N,,,0,40.0,...,1.187714,1,887.740871,0.0,16.807568,0.717029,1.906303,7.0,72572024127,1
14738364002,2,2016-09-14 07:00:00,7,2,9,N,,,0,40.0,...,1.187714,1,887.740871,0.0,16.807568,0.717029,1.906303,7.0,72572024127,1
13626864002,2,2013-03-07 20:00:00,20,3,3,N,,,0,40.0,...,1.187714,1,887.740871,0.0,16.807568,0.717029,1.906303,7.0,72572024127,1
14291928002,2,2015-04-16 14:00:00,14,3,4,N,,,0,40.0,...,1.187714,1,887.740871,0.0,16.807568,0.717029,1.906303,7.0,72572024127,1


# 4 Negative Sampling
We have a large set of positive examples, that is, when accidents occured. Any time/segment when an accident didn't occur is a valid negative sample. We sample possible negtive examples by sampling from and mutating the positive examples until they aren't positive. We do this by changing either the hour, day or segment ID.

In [18]:
class NegativeSampler:
    """
    Given features (positive examples), roads, times and the number of samples to draw, draws a sample
    """
    def __init__(self,num_samples,
                 accidents,roads,
                 static_feature_names,
                 seed=None):

        self.roads = roads
        self.accidents = accidents
        self.num_samples = num_samples
        
        self.static_feature_names = static_feature_names
        
        if seed:
            np.random.seed(seed)
    
    def sample(self):
        segment_ids = self.roads.index.to_series()
        
        altered = pd.DataFrame()
        num_to_sample = self.num_samples
        
        while num_to_sample > 0:
            samples = self.accidents[['timestamp','segment_id']].sample(n=num_to_sample,replace=True).reset_index()
            sample_timestamps,sample_segment_ids = self._mod(samples.copy(),segment_ids,1.0/(self.roads.accident_counts+1))
            # Create an index
            
            timestamp_strings = sample_timestamps.map(lambda x:str(int(x.timestamp()))).values
            segment_id_strings = sample_segment_ids.values.astype('str') 
        
            fid = timestamp_strings+segment_id_strings
            alt = pd.DataFrame({
                'timestamp': sample_timestamps.values,
                'segment_id': sample_segment_ids.values,
                'fid':fid
            }).set_index('fid')
            
            altered = altered.append(alt)

            # Which happened before? They shouldn't get negative samples
            intersection = altered.index.intersection(self.accidents.index)
            idxer = intersection.get_values()
            
            # Drop samples where accidents occurred.
            altered = altered.drop(idxer)
            
            num_to_sample = self.num_samples - altered.shape[0]
        
        
        altered['target'] = 0
        
        ts = pd.DatetimeIndex(altered.timestamp)
        segment_ids = altered.segment_id
        
        #altered = altered.reset_index()
        
        station_ids = self.roads.loc[segment_ids].station_id
        static_features = self.roads.loc[segment_ids][self.static_feature_names]
        for i,f in enumerate(self.static_feature_names):
            altered[f] = static_features.values[:,i]
        altered['station_id'] = station_ids.values
        altered['weekday'] = ts.weekday
        altered['month'] = ts.month
        altered['hour'] = ts.hour
        
        altered = altered.reset_index(drop=True)#.drop(columns=['fid'])
        return altered
    
    def _mod(self,samples,segment_ids,w=None):
        # Get the current timestamps
        ts = pd.DatetimeIndex(samples.timestamp)
        
        # Hour, Day, Year
        hour = ts.hour.to_series()
        day = ts.dayofyear.to_series()
        year = ts.year.to_series()

        # Road ID
        segment_id = samples.segment_id.copy()
        
        sample_segment_ids = pd.Series()
        sample_timestamps = pd.Series()
                                     
        # Index of samples to mutate
        #feat_i = np.random.randint(0,3,size=samples.shape[0])
        feat_i = np.random.choice([0,1,2],size=samples.shape[0],p=[0.1,0.3,0.6])
        ##########################
        # i == 0
        # Change hour of day
        idx = feat_i == 0
        samp_i = samples.loc[idx]
        N = samp_i.shape[0]

        # Sample until we have all different hours of the day
        num_same = N
        new_hours = hour.loc[idx].values.copy()
        dif_idx = np.ones(N,dtype='bool')
        
        while num_same != 0:
            
            new_hours[dif_idx] = np.random.choice(24,size=num_same)
            dif_idx = new_hours == hour.loc[idx].values
            dif_idx = dif_idx | ((new_hours - 1) == hour.loc[idx].values)
            dif_idx = dif_idx | ((new_hours + 1) == hour.loc[idx].values)
            num_same = dif_idx.sum()
       
        # Create new timestamps
        new_timestamps = year[idx].apply(pd.Timestamp,args=(1,1))
        new_timestamps = pd.DatetimeIndex(new_timestamps) 
        new_timestamps += pd.TimedeltaIndex(day[idx]-1,unit='D') # same day
        new_timestamps += pd.TimedeltaIndex(new_hours,unit='H') # new hour
        
        sample_segment_ids = sample_segment_ids.append(segment_id.loc[idx].copy())
        sample_timestamps = sample_timestamps.append(new_timestamps.to_series().copy())
        ##########################

        ##########################
        # i == 1
        # Change day of year
        idx = feat_i == 1
        samp_i = samples.loc[idx]
        N = samp_i.shape[0]

        is_leap_yr = ts[idx].is_leap_year

        # Sample until we have all different days of the year.
        num_same = N
        new_days = day.loc[idx].values.copy()
        dif_idx = np.ones(N,dtype='bool')
        while num_same != 0:
            # Pay attention to leap years
            dif_leap_yr = (dif_idx&is_leap_yr)
            dif_no_leap_yr = dif_idx&(~is_leap_yr)
            
            new_days[dif_leap_yr] = np.random.choice(np.arange(1,367,dtype='int'),size=dif_leap_yr.sum())
            new_days[dif_no_leap_yr] = np.random.choice(np.arange(1,366,dtype='int'),size=dif_no_leap_yr.sum())

            dif_idx = new_days == day.loc[idx].values
            num_same = dif_idx.sum()
        
        # Create new timestamps
        timestamps = year[idx].apply(pd.Timestamp,args=(1,1))
        new_timestamps = pd.DatetimeIndex(timestamps) 
        new_timestamps += pd.TimedeltaIndex(new_days-1,unit='D')  # new day
        new_timestamps += pd.TimedeltaIndex(hour[idx],unit='H') # same hour

        sample_segment_ids = sample_segment_ids.append(segment_id.loc[idx].copy())
        sample_timestamps = sample_timestamps.append(new_timestamps.to_series().copy())
        ##########################

        ##########################
        # i == 2
        # Change road
        idx = feat_i == 2
        samp_i = samples.loc[idx]
        N = samp_i.shape[0]

        num_same = N
        new_roads = segment_id.loc[idx].values.copy()
        dif_idx = np.ones(N,dtype='bool')

        while num_same != 0:
            new_roads[dif_idx] = segment_ids.sample(n=num_same,replace=True,weights=w).values
            dif_idx = new_roads == segment_id.loc[idx].values
            num_same = dif_idx.sum()
        
        sample_segment_ids = sample_segment_ids.append(pd.Series(new_roads))
        sample_timestamps = sample_timestamps.append(ts[idx].to_series().copy())
        
        
        sample_segment_ids = sample_segment_ids.astype('int64')     
        return sample_timestamps,sample_segment_ids

In [19]:
t = time.time()
N = features.shape[0]*6
ns = NegativeSampler(N,features,rdf,static_feature_names)

negative_examples = ns.sample().set_index(['timestamp','segment_id','station_id'])

print((time.time() - t),'seconds')

2272
0
126.26814579963684 seconds


In [20]:
negative_examples.to_csv('training_data/utah_negative_examples.csv')

In [21]:
negative_examples.head(50)

,,,target,pre_dir,street_type,suf_dir,one_way,speed_limit,surface_type,surface_width,aadt,sinuosity,...,at_intersection,proximity_to_billboard,proximity_to_major_road,proximity_to_signal,proximity_to_nearest_intersection,population_density,accident_counts,weekday,month,hour
timestamp,segment_id,station_id,,,,,,,,,,,,,,,,,,,,,
2016-09-21 04:00:00,37485,72479694128,0,S,ST,,0,40,,0,14458,1,...,1,1124.38,0,294.857,0.650994,0.110702,21,2,9,4
2011-04-11 21:00:00,29793,9999994133,0,S,,,0,65,PAVED,0,2154,1.00999,...,1,3286.28,0,9220.87,3.86581,0.00561377,30,0,4,21
2012-12-29 12:00:00,1934,72572024127,0,W,,S,0,35,,0,39196,1,...,1,847.998,0,18.2434,3.11272,2.37102,58,5,12,12
2011-07-11 07:00:00,42249,72572024127,0,S,FWY,,2,65,,0,10103,1.00127,...,0,113.966,0,1282.24,1325.4,0.628518,157,0,7,7
2012-10-25 12:00:00,24528,72572024127,0,,,,1,40,,0,1070,1.01195,...,0,2224.8,0,537.989,126.286,0.732324,9,3,10,12
2015-04-16 14:00:00,42111,72572024127,0,,FWY,,2,65,,0,1812,1.041,...,0,400.953,0,663.41,822.579,0.897667,134,3,4,14
2012-05-13 00:00:00,57924,72572024127,0,S,DR,,0,25,,0,0,1.00018,...,0,563.585,158.324,632.262,600.591,1.16747,1,6,5,0
2017-11-24 10:00:00,28671,72572024127,0,S,ST,,0,40,,0,26341,1.00026,...,1,173.024,0,17.2998,5.13366,2.15485,119,4,11,10
2011-10-31 10:00:00,168846,72572024127,0,S,,E,0,25,,0,0,1,...,0,1549.61,560.211,536.804,563.332,0.601587,4,0,10,10


In [22]:
# Reselect Features
positive_examples = df\
    .xs(['segment_id']+temporal_feature_names,axis=1)\
    .set_index('segment_id')\
    .join(
        rdf.xs(static_feature_names+['station_id'],axis=1),
        how='left'
    ).reset_index()
    
positive_examples = positive_examples.set_index([time_index.floor('1h'),'segment_id','station_id'])
positive_examples['target'] = 1
positive_examples.head()

,,,hour,weekday,month,pre_dir,street_type,suf_dir,one_way,speed_limit,surface_type,surface_width,...,segment_length,road_orient_approx,at_intersection,proximity_to_billboard,proximity_to_major_road,proximity_to_signal,proximity_to_nearest_intersection,population_density,accident_counts,target
timestamp,segment_id,station_id,,,,,,,,,,,,,,,,,,,,,
2016-11-26 10:00:00,1,72572024127,10,5,11,N,,,0,40.0,,0,...,110.198977,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,1
2017-10-16 16:00:00,1,72572024127,16,0,10,N,,,0,40.0,,0,...,110.198977,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,1
2010-01-16 15:00:00,1,72572024127,15,5,1,N,,,0,40.0,,0,...,110.198977,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,1
2013-12-16 15:00:00,1,72572024127,15,0,12,N,,,0,40.0,,0,...,110.198977,1.022803,1,220.261693,0.0,108.225287,1.578356,2.071348,4.0,1
2015-09-04 21:00:00,2,72572024127,21,4,9,N,,,0,40.0,,0,...,87.835598,1.187714,1,887.740871,0.0,16.807568,0.717029,1.906303,7.0,1


In [23]:
positive_examples.to_csv('training_data/utah_positive_examples.csv')

In [24]:
rdf.to_csv('training_data/road_features.csv')
df.to_csv('training_data/collisions.csv')

# Join Hourly Weather Data

In [25]:
pdf = pd.read_csv('training_data/utah_positive_examples.csv')
ndf = pd.read_csv('training_data/utah_negative_examples.csv')
wdf = pd.read_csv('utah_weather_2010-2018_grouped.csv')

In [26]:
pdf['timestamp'] = pd.DatetimeIndex(pd.to_datetime(pdf.timestamp))\
    .tz_localize('US/Mountain',ambiguous='NaT',errors='coerce').tz_convert('utc')
ndf['timestamp'] = pd.DatetimeIndex(pd.to_datetime(ndf.timestamp))\
    .tz_localize('US/Mountain',ambiguous='NaT',errors='coerce').tz_convert('utc')
wdf['timestamp'] = pd.DatetimeIndex(pd.to_datetime(wdf.timestamp))\
    .tz_localize('utc')

In [27]:
def add_join_key(df):
    df['join_key'] = df.station_id.map(int).map(str)+df.timestamp.map(datetime.isoformat)
    df = df.set_index('join_key')
    return df

In [28]:
pdf = add_join_key(pdf)
ndf = add_join_key(ndf)
wdf = add_join_key(wdf)
pdf = pdf.join(wdf.drop(columns=['timestamp', 'station_id']))
ndf = ndf.join(wdf.drop(columns=['timestamp', 'station_id']))

# Append negative samples to positive ones
tset = pdf.append(ndf)

In [29]:
# Calculate Solar Az/El
a = Astral()
a.solar_depression = 'civil'
SLC  = a['Salt Lake City']

def solar_az(x):
    try:
        return SLC.solar_azimuth(x)
    except:
        return np.nan
def solar_el(x):
    try:
        return SLC.solar_elevation(x)
    except:
        return np.nan    

   
tset['solar_azimuth'] = pd.DatetimeIndex(tset.timestamp).map(solar_az)
tset['solar_elevation'] = pd.DatetimeIndex(tset.timestamp).map(solar_el)

In [32]:
tset.to_csv('training_data/utah_training_set.csv')

In [31]:
tset.head()

,aadt,accident_counts,at_intersection,euclidean_length,foggy,hailing,hour,icy,month,one_way,...,surface_width,target,temperature,thunderstorm,timestamp,visibility,weekday,wind_speed,solar_azimuth,solar_elevation
join_key,,,,,,,,,,,,,,,,,,,,,
724700931412010-01-01T09:00:00+00:00,2582.0,2.0,1,183.940054,0.0,0.0,2,0.0,1,0,...,0,1,-7.2,0.0,2010-01-01 09:00:00+00:00,16093.0,4,0.0,53.133310,-64.221430
724700931412010-01-01T18:00:00+00:00,2608.0,11.0,0,470.656666,0.0,0.0,11,0.0,1,0,...,50,1,-5.0,0.0,2010-01-01 18:00:00+00:00,16093.0,4,0.0,157.114449,22.926000
724700931412010-01-02T09:00:00+00:00,6644.0,29.0,1,3445.573327,0.0,0.0,2,0.0,1,0,...,50,1,-13.3,0.0,2010-01-02 09:00:00+00:00,16093.0,5,0.0,52.812066,-64.224402
724700931412010-01-03T01:00:00+00:00,7849.0,29.0,1,1949.083740,0.0,0.0,18,0.0,1,0,...,0,1,-11.1,0.0,2010-01-03 01:00:00+00:00,16093.0,5,2.1,247.537151,-8.984745
724700931412010-01-03T15:00:00+00:00,6215.0,31.0,1,1983.762463,0.0,0.0,8,0.0,1,0,...,0,1,-8.9,0.0,2010-01-03 15:00:00+00:00,16093.0,6,0.0,121.156632,0.908545
